In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
from attgcn_api import ATTGCN_API
import numpy as np
from py_utils.TSP_loader import TSP_loader

In [3]:
def create_dir(save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

In [4]:

gcn_api = ATTGCN_API()
gcn_api.init_net()
gcn_api.load_ckpt()

Using CUDA!


In [5]:
NUM_NODES = 20
n_graphs = 50000
path = f'./data/train_sets'
folder = f'synthetic_n_{NUM_NODES}_{n_graphs}'

In [6]:
tsp_loader = TSP_loader()

In [7]:
for i in range(50):
    g_list = tsp_loader.load_multi_tsp_as_nx(data_dir=f'{path}/{folder}', scale_factor=10**(-6), start_index=i*1000, end_index=(i+1)*1000)
    gcn_api.load_nx_test_set(graph_list=g_list, num_nodes=20)
    result = gcn_api.run_test(batch_size=250)
    save_path = f'{path}/{folder}/heatmaps'
    create_dir(save_path)
    t = i * 1000
    file_name = f'{folder}_heatmaps_{t}-{t+1000}.npy'
    np.save(file=f'{save_path}/{file_name}', arr=np.array(result))

 75%|███████▌  | 3/4 [00:04<00:01,  1.34s/it]